# Azure Open AI Basics

## Test an Azure OpenAI Deployment

Here I show how to use the OpenAI deployment programmatically via REST; I used the API key and the Endpoint obtained from the Azure AI Foundry (Deployments).
Also, in the Chat Playground, if we click on `View code` we see the Python code for interaction, which is different to this one.
The example code from `View code` has more options!

![Azure AI Foundry: Deployments](../assets/auzure_ai_foundry_deployments.png)
![Azure AI Foundry: Playground](../assets/auzure_ai_foundry_playground.png)

### Simple API Call

In [3]:
import os
import requests
from dotenv import load_dotenv

load_dotenv(override=True, dotenv_path=".env")

True

In [4]:
# Define your Azure OpenAI details
AZURE_OPENAI_ENDPOINT_URI = os.getenv("AZURE_OPENAI_ENDPOINT_URI")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")

In [8]:
# Set up the request headers
headers = {
    "Content-Type": "application/json",
    "api-key": AZURE_OPENAI_API_KEY,
}

# Define the request body
data = {
    "messages": [
        {"role": "system", "content": "You are an assistant."},
        {"role": "user", "content": "Tell me a joke."},
    ],
    "max_tokens": 100,
}

# Send the POST request
response = requests.post(
    f"{AZURE_OPENAI_ENDPOINT_URI}",
    headers=headers,
    json=data,
)

# Handle the response
if response.status_code == 200:
    result = response.json()
    print("Response:")
    print(result["choices"][0]["message"]["content"])
else:
    print(f"Error: {response.status_code} - {response.text}")

Response:
Why don't scientists trust atoms? 

Because they make up everything.


### Code Example from Playground

In [11]:
import os  
import base64
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv(override=True, dotenv_path=".env")

True

In [12]:
# Now, the URL is shorter and we add the endpoint and deployment name
endpoint = os.getenv("ENDPOINT_URL")
deployment = os.getenv("DEPLOYMENT_NAME")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")

In [13]:
# Initialize Azure OpenAI client with key-based authentication    
client = AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version="2024-05-01-preview",  
)

In [15]:
#Prepare the chat prompt 
chat_prompt = [
    {
        "role": "system",
        "content": "You are an AI assistant that helps people find information."
    }
] 
    
# Include speech result if speech is enabled  
messages = chat_prompt  
    
# Generate the completion  
completion = client.chat.completions.create(  
    model=deployment,  
    messages=messages,  
    max_tokens=800,  
    temperature=0.7,  
    top_p=0.95,  
    frequency_penalty=0,  
    presence_penalty=0,  
    stop=None,  
    stream=False
)

print(completion.to_json())

{
  "id": "chatcmpl-Alilpzix5RQAIL0QpwOn51NIpD9py",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello! How can I assist you in finding information today?",
        "role": "assistant"
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1735936877,
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 12,
    "prompt_tokens": 19,
    "total_tokens": 31
  },
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {}
    

### Chat Loop with Playground Code

In [34]:
import os  
import base64
from openai import AzureOpenAI
from openai import ChatCompletion
from dotenv import load_dotenv
from typing import List, Dict, Optional

load_dotenv(override=True, dotenv_path=".env")

True

In [17]:
# Now, the URL is shorter and we add the endpoint and deployment name
endpoint = os.getenv("ENDPOINT_URL")
deployment = os.getenv("DEPLOYMENT_NAME")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")

In [18]:
# Initialize Azure OpenAI client with key-based authentication    
client = AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version="2024-05-01-preview",  
)

In [38]:
def chat(user_message: str, history: Optional[list[dict]] = None) -> ChatCompletion:
    if history is None:
        history = [
            {
                "role": "system",
                "content": "You are an AI assistant that helps people find information."
            }
        ]

    # Insert the user message into the history
    history.append(
        {
            "role": "user",
            "content": user_message
        }
    )
    
    # Generate the completion  
    completion = client.chat.completions.create(  
        model=deployment,  
        messages=history,  
        max_tokens=800,  
        temperature=0.7,  
        top_p=0.95,  
        frequency_penalty=0,  
        presence_penalty=0,  
        stop=None,  
        stream=False
    )
    
    # Extend history
    history.append(
        {
            "role": "assistant",
            "content": completion.choices[0].message.content
        }
    )
    
    return completion, history

In [44]:
completion, history = chat("Tell me a joke.")
print(completion.to_dict()["choices"][0]["message"]["content"])

Why did the tomato turn red? Because it saw the salad dressing!
[{'role': 'system', 'content': 'You are an AI assistant that helps people find information.'}, {'role': 'user', 'content': 'Tell me a joke.'}, {'role': 'assistant', 'content': 'Why did the tomato turn red? Because it saw the salad dressing!'}]


In [ ]:
history = None
while True:
    user_message = input(">>: ")
    completion, history = chat(user_message, history)
    print(f"assistant: {completion.to_dict()['choices'][0]['message']['content']}")

assistant: Sure, here's a joke for you: Why don't scientists trust atoms? Because they make up everything.
assistant: Sure, here's an animal joke for you: What do you call a grizzly bear with no teeth? A gummy bear!
assistant: As an AI language model, I don't have the ability to have preferences or emotions. However, I am designed to generate different types of jokes based on the user's request.
assistant: Sure, here is a summary of our conversation so far:

- You asked me to tell you a joke, so I told you a joke about atoms.
- You then asked me to tell you a joke about animals, so I told you a joke about a grizzly bear with no teeth.
- Lastly, you asked me which joke I prefer, but as an AI language model, I don't have the ability to have preferences or emotions.
assistant: Is there anything else I can help you with?
assistant: I'm sorry, I didn't receive a question or request from you. Is there something specific you need assistance with?
assistant: If you have any questions or need a